In [12]:
import torch.nn as nn
import torch
import torch.optim as optim

In [13]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [14]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [15]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [16]:
t_un_val

tensor([[5.6300],
        [6.0400]])

In [17]:
linear_model=nn.Linear(1,1)
linear_model(t_un_val)


tensor([[5.9201],
        [6.3134]], grad_fn=<AddmmBackward0>)

In [18]:
linear_model.weight

Parameter containing:
tensor([[0.9592]], requires_grad=True)

In [19]:
linear_model.bias

Parameter containing:
tensor([0.5199], requires_grad=True)

In [20]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
         # <2>
        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")


In [21]:
linear_model=nn.Linear(1,1)
optimizer=optim.SGD(linear_model.parameters(),lr=1e-2)


In [22]:
training_loop(n_epochs=3000,optimizer=optimizer,model=linear_model,loss_fn=nn.MSELoss(),t_u_train=t_un_train,t_u_val=t_un_val,t_c_train=t_c_train,t_c_val=t_c_val)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 127.6511, Validation loss 74.4688


Epoch 1000, Training loss 2.9786, Validation loss 6.1061
Epoch 2000, Training loss 2.4374, Validation loss 6.5092
Epoch 3000, Training loss 2.4288, Validation loss 6.5618

Parameter containing:
tensor([[5.4814]], requires_grad=True)
Parameter containing:
tensor([-17.4256], requires_grad=True)
